# Creating the web scraper

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

Traceback (most recent call last):
  File "C:\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Python310\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


ModuleNotFoundError: No module named 'bs4'

# Getting the dataset

In [24]:
data = pd.read_csv('../data/music-subset.csv')

In [25]:
refined_data = data.loc[(data["rank"] == 1)]

In [21]:
# new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [22]:
new_data = refined_data.reset_index(drop = True)

In [23]:
refined_data.shape

(6, 22)

In [11]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,195840.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,259196.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,222560.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,205600.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,234320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Tim Tim - Ao Vivo,126,2017-01-01,Wesley Safadão,https://open.spotify.com/track/4bftZ7IayYg0I6q...,Brazil,top200,MOVE_UP,37363.0,...,2.0,-4.964,0.0,0.0504,0.58100,0.000000,0.1430,0.891,163.998,174067.0
996,996,Folgado - Ao Vivo,127,2017-01-01,Marília Mendonça,https://open.spotify.com/track/4DYmJ19DhmxUe26...,Brazil,top200,MOVE_UP,37320.0,...,1.0,-3.792,0.0,0.0729,0.64500,0.000000,0.0726,0.569,163.848,163902.0
997,997,Photograph,128,2017-01-01,Ed Sheeran,https://open.spotify.com/track/6fxVffaTuwjgEk5...,Brazil,top200,MOVE_DOWN,37266.0,...,4.0,-10.480,1.0,0.0476,0.60700,0.000464,0.0986,0.201,107.989,258987.0
998,998,Starving,129,2017-01-01,"Hailee Steinfeld, Grey, Zedd",https://open.spotify.com/track/1qOLh0tI7trd1zd...,Brazil,top200,MOVE_DOWN,37178.0,...,4.0,-4.230,1.0,0.1260,0.37100,0.000000,0.1020,0.512,99.721,181880.0


In [14]:
new_data.drop(index_names, inplace=True)

In [13]:
new_data.shape

(6, 22)

In [30]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0
25,Rockabye (feat. Sean Paul & Anne-Marie),1,2017-01-01,Clean Bandit,https://open.spotify.com/track/5knuzwU65gJK7IF...,Hungary,top200,MOVE_UP,6263.0
...,...,...,...,...,...,...,...,...,...
227143,Diamond in the rough,1,2021-07-30,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,Iceland,viral50,MOVE_UP,NaN
227144,Aadmi Chutiya Hai,1,2021-07-30,Rahgir,https://open.spotify.com/track/2PEJKORCdmceTW2...,India,viral50,MOVE_UP,NaN
227196,Bezos I,1,2021-07-31,Bo Burnham,https://open.spotify.com/track/0Kdj7nwaYQmvhxn...,Czech Republic,viral50,MOVE_UP,NaN
227202,Paar tundi veel,1,2021-07-31,Shanon,https://open.spotify.com/track/3LM9EDiw6CtKhCt...,Estonia,viral50,MOVE_UP,NaN


In [17]:
contain_values = data[data.artist.str.contains('The Weeknd')]
print (contain_values)

     index                 title  rank        date  \
28      28               Starboy    29  2017-01-01   
75      75      I Feel It Coming    76  2017-01-01   
201    201               Starboy     1  2017-01-01   
207    207      I Feel It Coming     7  2017-01-01   
266    266         Party Monster    66  2017-01-01   
307    307               Rockin’   107  2017-01-01   
331    331    Can't Feel My Face   130  2017-01-01   
333    333             Sidewalks   132  2017-01-01   
368    368  Starboy - Kygo Remix   167  2017-01-01   
388    388        Six Feet Under   187  2017-01-01   
394    394             The Hills   193  2017-01-01   
408    408               Starboy     7  2017-01-01   
449    449      I Feel It Coming    47  2017-01-01   
560    560         Party Monster   158  2017-01-01   
605    605               Starboy     3  2017-01-01   
621    621      I Feel It Coming    19  2017-01-01   
725    725         Party Monster   123  2017-01-01   
819    819               Sta

In [13]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [18]:
new_data.head()

,index,title,rank,date,artist,url,region,chart,trend,streams,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,201,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0,...,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453.0
2,402,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0,...,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.121,0.157,97.019,161200.0
3,603,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0,...,6.0,-4.582,0.0,0.0845,0.0154,0.000000,0.235,0.831,90.018,190813.0


In [15]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [19]:
new_data.shape

(3, 22)

In [17]:
new_data.iloc[140087:140100,:]

,title,rank,date,artist,url,region,chart,trend,streams


In [18]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0


# Import dataset

In [94]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [298]:
data.head(40)

,title,rank,date,artist,url,region,chart,trend,streams,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,0.852,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.9070,102.034,195840.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,0.663,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.5330,99.935,259196.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,0.761,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.7100,93.974,222560.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,0.508,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.5550,180.044,205600.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,0.899,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.8730,88.007,234320.0
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spotify.com/track/5J1c3M4EldCfNxX...,Argentina,top200,MOVE_DOWN,151140.0,0.776,...,11.0,-4.933,1.0,0.0638,0.14200,0.000000,0.2190,0.6610,91.012,228467.0
6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,https://open.spotify.com/track/1MpKZi1zTXpERKw...,Argentina,top200,MOVE_DOWN,148369.0,0.588,...,11.0,-7.169,0.0,0.1730,0.08510,0.000027,0.0840,0.9370,205.643,195274.0
7,Otra vez (feat. J Balvin),8,2017-01-01,Zion & Lennox,https://open.spotify.com/track/3QwBODjSEzelZyV...,Argentina,top200,MOVE_DOWN,143004.0,0.832,...,10.0,-5.429,1.0,0.1000,0.05590,0.000486,0.4400,0.7040,96.016,209453.0
8,La Bicicleta,9,2017-01-01,"Carlos Vives, Shakira",https://open.spotify.com/track/0sXvAOmXgjR2QUq...,Argentina,top200,MOVE_UP,126389.0,0.736,...,0.0,-2.147,1.0,0.1290,0.19800,0.000002,0.3360,0.9530,179.935,227707.0


# Extracting artist and track name

In [324]:
track_nr = 449

In [256]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [325]:
get_track_name(track_nr)

'I Feel It Coming'

In [326]:
get_track_artist(track_nr)

'The Weeknd'

# Getting URL

In [274]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [327]:
get_url(track_nr)

'https://genius.com/The-Weeknd-I-Feel-It-Coming-lyrics'

# Scraping credits

In [105]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [328]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))

# Dict to csv

In [329]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

In [330]:
df = pd.DataFrame.from_dict(credit_list) 
df.to_csv (r'C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv', index = False, header=True)

In [331]:
df

,Feature,Writer,Label,Producer
0,Daft Punk,The Weeknd,XO Records,Daft Punk
1,None,Rico The Wizard,Universal Music Group,Cirkut
2,None,Guy-Manuel de Homem-Christo,Republic Records,Doc McKinney
3,None,Thomas Bangalter,None,The Weeknd
4,None,Doc McKinney,None,None
5,None,Cirkut,None,None


***

# Neo4j

In [110]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [332]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test123")


In [56]:
conn.query('''MATCH (n) RETURN n;''')

[]

In [333]:
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'I Feel It Coming'

In [334]:
artist

'The Weeknd'

### Query to insert into neo4j

In [296]:
def run_querys(a, t):
    
    artist = a
    track = t
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [335]:
run_querys(artist, track)

In [267]:
df

,Feature,Writer,Label,Producer
0,None,Nicky Jam,Sony Music Entertainment Colombia S.A,Chan El Genio
1,None,Kevin ADG,Sony Music Latin,Kevin ADG
2,None,Chan El Genio,Diego Abaroa,None
3,None,Bull Nene,None,None
4,None,Maluma,None,None
